In [116]:
from simpleai.search import SearchProblem
import random

GOAL = (('1', '2', '3'),
        ('8', '@', '4'),
        ('7', '6', '5'))
        
initial_state = (('8', '1', '2'),
                 ('6', '5', '3'),
                 ('@', '7', '4'))

def AdjustState(state):
    state1d = Change2Dto1D(state)
    stateset = set(state1d)
    missingset = set(['@','1','2','3','4','5','6','7','8'])
    missingset.difference_update(stateset)
    remain = list(missingset)
    for i in range(len(state1d) - 1):
        for j in range(i + 1, len(state1d)):
            if state1d[i] == state1d[j]:
                state1d[j] = remain[0]
                remain.remove(remain[0])
                break
    return Change1Dto2D(state1d)

def Change1Dto2D(oneD_array):
    rows = len(oneD_array) // 3 
    return [oneD_array[i * 3:(i + 1) * 3] for i in range(rows)]

def Change2Dto1D(twoD_array):
    return [item for row in twoD_array for item in row]

def Convert2DListToString(input_list):
    return "\n".join([" ".join(str(x) for x in row) for row in input_list])

class PuzzleProblem(SearchProblem):
    def value(self, state):
        sum = 0
        for i in range(3):
            for j in range(3):
                if state[i][j] == GOAL[i][j]:
                    sum += 1
        return sum
    
    def actions(self, state):
        actions = []
        empty_row, empty_col = [(r, c) for r, row in enumerate(state) for c, val in enumerate(row) if val == '@'][0]

        #Tập hợp các quy tắt tuongw ứng với từng UDLR
        moves = {'Up': (empty_row - 1, empty_col),
                 'Down': (empty_row + 1, empty_col),
                 'Left': (empty_row, empty_col - 1),
                 'Right': (empty_row, empty_col + 1)}

        #Kiểm tra các hành đônng khả khi
        for action, (row, col) in moves.items():
            if 0 <= row <= 2 and 0 <= col <= 2:
                actions.append(action)
        random.shuffle(actions)
        return actions

    def result(self, state, action):
        empty_row, empty_col = [(r, c) for r, row in enumerate(state) for c, val in enumerate(row) if val == '@'][0]
        new_state = [list(row) for row in state]  # Tạo bản sao của trạng thái

        if action == 'Up':
            target_row, target_col = empty_row - 1, empty_col
        elif action == 'Down':
            target_row, target_col = empty_row + 1, empty_col
        elif action == 'Left':
            target_row, target_col = empty_row, empty_col - 1
        elif action == 'Right':
            target_row, target_col = empty_row, empty_col + 1
        
        new_state[empty_row][empty_col], new_state[target_row][target_col] = new_state[target_row][target_col], new_state[empty_row][empty_col]
        return tuple(tuple(row) for row in new_state)

    def generate_random_state(self):
        # flattened_goal = sum(GOAL, ())
        # random_state = list(flattened_goal)
        # random.shuffle(random_state)
        # return tuple(tuple(random_state[i:i+3]) for i in range(0, 9, 3))
        orin = ['1', '2', '3', '4', '5', '6', '7', '8', '@']
        random.shuffle(orin)
        init_state = Change1Dto2D(orin)
        return init_state
    
    def crossover(self, state1, state2):
        cut_first = state1[0][:]
        cut_second = state2[1:3][:]
        union = [cut_first, cut_second[0], cut_second[1]]
        union = AdjustState(union)
        return union
    
    def mutate(self, state):
        i, j = random.randint(0, 2), random.randint(0, 2)
        mutated = state.copy()
        if i == j:
            i = (i + 1) % 3
        temp = mutated[i][j]
        mutated[i][j] = mutated[j][i]
        mutated[j][i] = temp
        return mutated
    
    def is_goal(self, state):
        return state == GOAL

    def cost(self, state1, action, state2):
        return 1 
    def heuristic(self, state):
        wrong = 0
        for i in range(len(state)):
            for j in range(len(state[i])):
                if state[i][j] != '@' and state[i][j] != GOAL[i][j]:
                    wrong+=1
        return wrong

    def print_state(self, state):
        return Convert2DListToString(state)
problem = PuzzleProblem(initial_state)

In [163]:
from simpleai.search.local import hill_climbing

problem = PuzzleProblem(initial_state)

result = hill_climbing(problem)
for i in range(100000):
    result = hill_climbing(problem)
    if result.state == GOAL:
        break
print(problem.print_state(result.state))

KeyboardInterrupt: 

In [162]:
from simpleai.search.local import simulated_annealing
result = simulated_annealing(problem)
print(problem.print_state(result.state))

8 1 2
6 5 3
@ 7 4


In [132]:
from simpleai.search.local import genetic

result = genetic(problem, population_size=100, iterations_limit=50, mutation_chance=0.1)
print(problem.print_state(result.state))

1 2 3
8 @ 4
7 6 5


In [41]:
from simpleai.search import SearchProblem

GOAL = (('1', '2', '3'),
        ('8', '@', '4'),
        ('7', '6', '5'))
        
initial_state = (('8', '1', '2'),
                 ('6', '5', '3'),
                 ('@', '7', '4'))

class PuzzleProblem(SearchProblem):
    def actions(self, state):
        actions = []
        empty_row, empty_col = [(r, c) for r, row in enumerate(state) for c, val in enumerate(row) if val == '@'][0]

        #Tập hợp các quy tắt tuongw ứng với từng UDLR
        moves = {'Up': (empty_row - 1, empty_col),
                 'Down': (empty_row + 1, empty_col),
                 'Left': (empty_row, empty_col - 1),
                 'Right': (empty_row, empty_col + 1)}

        #Kiểm tra các hành đônng khả khi
        for action, (row, col) in moves.items():
            if 0 <= row <= 2 and 0 <= col <= 2:
                actions.append(action)
        return actions

    def result(self, state, action):
        empty_row, empty_col = [(r, c) for r, row in enumerate(state) for c, val in enumerate(row) if val == '@'][0]
        new_state = [list(row) for row in state]  # Tạo bản sao của trạng thái

        #Thực hiện hành động di chuyển để giải với că cú theo UDLR gán cho biến mới và sau đó trả về trạng thái mới với vịt trí mới
        if action == 'Up':
            target_row, target_col = empty_row - 1, empty_col
        elif action == 'Down':
            target_row, target_col = empty_row + 1, empty_col
        elif action == 'Left':
            target_row, target_col = empty_row, empty_col - 1
        elif action == 'Right':
            target_row, target_col = empty_row, empty_col + 1
        
        new_state[empty_row][empty_col],new_state[target_row][target_col] = new_state[target_row][target_col],new_state[empty_row][empty_col]
        return tuple(tuple(row) for row in new_state)

    def is_goal(self, state):
        return state == GOAL

    def cost(self, state1, action, state2):
        return 1 
    def heuristic(self, state):
        # how far are we from the goal?
        wrong = sum([1 if state[i] != GOAL[i] else 0
                    for i in range(len(state))])
        missing = len(GOAL) - len(state)
        return wrong + missing

    def print_state(self, state):
        for row in state:
            print(' '.join(row))
        print()
problem = PuzzleProblem(initial_state)

In [45]:
from simpleai.search import astar
result = astar(problem)
print("Start:")
print(result.path())
for action, state in result.path():
    print(f"Thực hiện hành động: {action}")
    problem.print_state(state)

Start:
[(None, (('2', '4', '3'), ('1', '@', '5'), ('8', '7', '6'))), ('Up', (('2', '@', '3'), ('1', '4', '5'), ('8', '7', '6'))), ('Left', (('@', '2', '3'), ('1', '4', '5'), ('8', '7', '6'))), ('Down', (('1', '2', '3'), ('@', '4', '5'), ('8', '7', '6'))), ('Down', (('1', '2', '3'), ('8', '4', '5'), ('@', '7', '6'))), ('Right', (('1', '2', '3'), ('8', '4', '5'), ('7', '@', '6'))), ('Right', (('1', '2', '3'), ('8', '4', '5'), ('7', '6', '@'))), ('Up', (('1', '2', '3'), ('8', '4', '@'), ('7', '6', '5'))), ('Left', (('1', '2', '3'), ('8', '@', '4'), ('7', '6', '5')))]
Thực hiện hành động: None
2 4 3
1 @ 5
8 7 6

Thực hiện hành động: Up
2 @ 3
1 4 5
8 7 6

Thực hiện hành động: Left
@ 2 3
1 4 5
8 7 6

Thực hiện hành động: Down
1 2 3
@ 4 5
8 7 6

Thực hiện hành động: Down
1 2 3
8 4 5
@ 7 6

Thực hiện hành động: Right
1 2 3
8 4 5
7 @ 6

Thực hiện hành động: Right
1 2 3
8 4 5
7 6 @

Thực hiện hành động: Up
1 2 3
8 4 @
7 6 5

Thực hiện hành động: Left
1 2 3
8 @ 4
7 6 5



In [44]:
from simpleai.search import greedy
initial_state =(('2', '4', '3'),
                ('1', '@', '5'),
                ('8', '7', '6'))
problem = PuzzleProblem(initial_state)

result = greedy(problem)
print("Start:")
print(result.path())
for action, state in result.path():
    print(f"Thực hiện hành động: {action}")
    problem.print_state(state)

Start:
[(None, (('2', '4', '3'), ('1', '@', '5'), ('8', '7', '6'))), ('Left', (('2', '4', '3'), ('@', '1', '5'), ('8', '7', '6'))), ('Down', (('2', '4', '3'), ('8', '1', '5'), ('@', '7', '6'))), ('Right', (('2', '4', '3'), ('8', '1', '5'), ('7', '@', '6'))), ('Right', (('2', '4', '3'), ('8', '1', '5'), ('7', '6', '@'))), ('Up', (('2', '4', '3'), ('8', '1', '@'), ('7', '6', '5'))), ('Left', (('2', '4', '3'), ('8', '@', '1'), ('7', '6', '5'))), ('Right', (('2', '4', '3'), ('8', '1', '@'), ('7', '6', '5'))), ('Left', (('2', '4', '3'), ('8', '@', '1'), ('7', '6', '5'))), ('Left', (('2', '4', '3'), ('@', '8', '1'), ('7', '6', '5'))), ('Right', (('2', '4', '3'), ('8', '@', '1'), ('7', '6', '5'))), ('Right', (('2', '4', '3'), ('8', '1', '@'), ('7', '6', '5'))), ('Up', (('2', '4', '@'), ('8', '1', '3'), ('7', '6', '5'))), ('Left', (('2', '@', '4'), ('8', '1', '3'), ('7', '6', '5'))), ('Right', (('2', '4', '@'), ('8', '1', '3'), ('7', '6', '5'))), ('Left', (('2', '@', '4'), ('8', '1', '3'), ('7